In [12]:
# dependencies
import pandas as pd
import optuna
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [13]:
dataset = pd.read_csv('TUANDROMD.csv')

dataset.info()
df = dataset.dropna() # cleaned dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4465 entries, 0 to 4464
Columns: 242 entries, ACCESS_ALL_DOWNLOADS to Label
dtypes: float64(241), object(1)
memory usage: 8.2+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 4464 entries, 0 to 4464
Columns: 242 entries, ACCESS_ALL_DOWNLOADS to Label
dtypes: float64(241), object(1)
memory usage: 8.3+ MB


In [14]:
# as described in uci website, Target Role is the attribute Label
X = df.drop(columns=['Label'], axis=1)  # Features
y = df['Label']  # Target variable
y.head()

0    malware
1    malware
2    malware
3    malware
4    malware
Name: Label, dtype: object

In [15]:
# Encode the categorical target variable using Label Encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_encoded = 1 - y_encoded
y_encoded

array([0, 0, 0, ..., 1, 1, 1])

In [16]:
# Split the dataset into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [17]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:

def objective(trial):
    C = trial.suggest_loguniform('C', 1e-3, 1)
    gamma = trial.suggest_loguniform('gamma', 1e-1, 1e1)

    # Create the SVM classifier with RBF kernel using suggested hyperparameters
    svm_classifier = SVC(kernel='rbf', C=C, gamma=gamma)

    # Use stratified k-fold cross-validation for better validation performance
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Calculate cross-validated accuracy with stratification
    cv_accuracy = cross_val_score(svm_classifier, X_train, y_train, cv=cv, scoring='accuracy').mean()

    print(cv_accuracy)
    return cv_accuracy

In [19]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100) # adjust n_trials

[I 2023-09-06 00:35:10,877] A new study created in memory with name: no-name-6c68070d-9eca-4e2f-98af-c8839214325f
/var/folders/gp/r_0zq_nx5pl3nmv_2z2bd6hc0000gn/T/ipykernel_46961/506725017.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  C = trial.suggest_loguniform('C', 1e-3, 1)
/var/folders/gp/r_0zq_nx5pl3nmv_2z2bd6hc0000gn/T/ipykernel_46961/506725017.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  gamma = trial.suggest_loguniform('gamma', 1e-1, 1e1)
[I 2023-09-06 00:35:13,531] Trial 0 finished with value: 0.9882396035337212 and parameters: {'C': 0.7665512987440601, 'gamma': 5.756763570299377}. Best is trial 0 with value: 0.98823960353

0.9882396035337212


/var/folders/gp/r_0zq_nx5pl3nmv_2z2bd6hc0000gn/T/ipykernel_46961/506725017.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  C = trial.suggest_loguniform('C', 1e-3, 1)
/var/folders/gp/r_0zq_nx5pl3nmv_2z2bd6hc0000gn/T/ipykernel_46961/506725017.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  gamma = trial.suggest_loguniform('gamma', 1e-1, 1e1)
[I 2023-09-06 00:35:17,691] Trial 1 finished with value: 0.8070580399992163 and parameters: {'C': 0.041172062612715374, 'gamma': 1.6647806427078062}. Best is trial 0 with value: 0.9882396035337212.


0.8070580399992163


/var/folders/gp/r_0zq_nx5pl3nmv_2z2bd6hc0000gn/T/ipykernel_46961/506725017.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  C = trial.suggest_loguniform('C', 1e-3, 1)
/var/folders/gp/r_0zq_nx5pl3nmv_2z2bd6hc0000gn/T/ipykernel_46961/506725017.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  gamma = trial.suggest_loguniform('gamma', 1e-1, 1e1)
[I 2023-09-06 00:35:21,790] Trial 2 finished with value: 0.8070580399992163 and parameters: {'C': 0.04534734539096853, 'gamma': 1.26221396702409}. Best is trial 0 with value: 0.9882396035337212.


...

0.9882396035337212


/var/folders/gp/r_0zq_nx5pl3nmv_2z2bd6hc0000gn/T/ipykernel_46961/506725017.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  C = trial.suggest_loguniform('C', 1e-3, 1)
/var/folders/gp/r_0zq_nx5pl3nmv_2z2bd6hc0000gn/T/ipykernel_46961/506725017.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  gamma = trial.suggest_loguniform('gamma', 1e-1, 1e1)
[I 2023-09-06 00:39:56,309] Trial 98 finished with value: 0.9680762374880022 and parameters: {'C': 0.5525512884185193, 'gamma': 6.973778227274633}. Best is trial 0 with value: 0.9882396035337212.


0.9680762374880022


/var/folders/gp/r_0zq_nx5pl3nmv_2z2bd6hc0000gn/T/ipykernel_46961/506725017.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  C = trial.suggest_loguniform('C', 1e-3, 1)
/var/folders/gp/r_0zq_nx5pl3nmv_2z2bd6hc0000gn/T/ipykernel_46961/506725017.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  gamma = trial.suggest_loguniform('gamma', 1e-1, 1e1)
[I 2023-09-06 00:39:59,323] Trial 99 finished with value: 0.9680762374880022 and parameters: {'C': 0.3709916094895886, 'gamma': 5.192395986025994}. Best is trial 0 with value: 0.9882396035337212.


0.9680762374880022


In [20]:
# Get the best hyperparameters
n_trials = len(study.trials)
best_params = study.best_params

best_C = best_params['C']
best_gamma = best_params['gamma']

svm_classifier = SVC(kernel='rbf', C=best_C, gamma=best_gamma)
svm_classifier = svm_classifier.fit(X_train, y_train)

In [21]:
y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")


Accuracy: 0.9932810750279956
Precision: 0.9728260869565217
Recall: 0.9944444444444445
F1-score: 0.9835164835164836


In [22]:
file_path = "model_evaluation.txt"

with open(file_path, 'a') as file:
    # Write the best hyperparameters
    file.write("Best Hyperparameters:\n")
    file.write(f"C: {best_C}\n")
    file.write(f"Gamma: {best_gamma}\n")
    file.write(f"n_trials: {n_trials}\n")
    
    # Write the evaluation scores
    file.write("\nEvaluation Scores:\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1-score: {f1}\n")
    file.write("\n")

print(f"Model evaluation information saved to {file_path}")

Model evaluation information saved to model_evaluation.txt
